In [ ]:
%load_ext autoreload
%autoreload 2

# Exploration des résultats du modèle

## Chargement des librairies, des données et du modèle

Attention à bien utiliser les mêmes hyperparamètres de modèle et noms de variables que lors de l'entraînement

In [ ]:
from pathlib import Path

import pandas as pd
import torch
import torch.nn as nn
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from reader import FromPandasDataset
from model import (
    PhysicalModel,
    FlightMecaNet3DEq,
    GenericFlightMecaDataset,
    GenericFlightMecaDatasetFast,
)
from reader.preprocessing import lbs_to_kg, deg_to_rad, celcius_to_kelvin, kts_to_ms
from visualisation import show_residue_evolution, show_all_anomaly_scores, show_anomaly_score_single_flight
from postprocessing.anomaly_score import add_anomaly_score

In [ ]:
training_name = "test_0"

time_var = "Time, s"
flight_name_var = "flight_name"
mass_var = "MASS"
jx_var = "JX"
jy_var = "JY"
jz_var = "JZ"
alpha_var = "ALPHA"
beta_var = "BETA"
pressure_var = "PST"
temp_var = "SAT"
air_speed_var = "VTAS"
mach_var = "MACH"
gear_var = "MLG"
flaps_bool_var = "DB"
flaps_var = "DVOLIG"
stab_var = "TRIM"
elevator_var = "DM"
rudder_var = "DN"
aileron_var = "DL"
spoiler_var = "DSPOIL"
n1_var = "N1"
altitude_var = "ZBPIL"
roll_angle_var = "PHI"

cx_input_vars = [
    alpha_var,
    flaps_var,
    stab_var,
    elevator_var,
    mach_var,
    gear_var,
]

cy_input_vars = [
    beta_var,
    rudder_var,
    mach_var,
]

cz_input_vars = [
    alpha_var,
    flaps_var,
    stab_var,
    elevator_var,
    mach_var,
    gear_var,
]

thrust_input_vars = [
    n1_var,
    pressure_var,
    temp_var,
]

In [ ]:
datapath = Path.cwd() / "test_data" / "test_data.csv"

df = pd.read_csv(datapath)
df = df.set_index([flight_name_var, time_var])

In [ ]:
df = lbs_to_kg(df, [mass_var])
df = deg_to_rad(
    df,
    [
        alpha_var,
        beta_var,
        elevator_var,
        rudder_var,
        aileron_var,
        spoiler_var,
        roll_angle_var,
    ],
)
df = celcius_to_kelvin(df, [temp_var])
df = kts_to_ms(df, [air_speed_var])

In [ ]:
to_normalize = cx_input_vars + cy_input_vars + cz_input_vars + thrust_input_vars
to_normalize = list(set(to_normalize))

# train_flight_names = ["vol 1", "vol 2", "vol 3", "vol 4", "vol 5", "vol 6", "vol 7", "vol 8"]
train_flight_names = None
data = FromPandasDataset(
    df,
    train_flight_names=train_flight_names,
    data_reduction=None,
    to_normalize=to_normalize,
    filter_train_phases=True,
)

In [ ]:
dataset_var_names_list = [
    cx_input_vars,
    cy_input_vars,
    cz_input_vars,
    thrust_input_vars,
]
dataset_var_names = [
    mass_var,
    jx_var,
    jy_var,
    jz_var,
    alpha_var,
    beta_var,
    pressure_var,
    temp_var,
    air_speed_var,
]
dataset_params = {
    "var_names": dataset_var_names,
    "var_names_list": dataset_var_names_list,
}


net_coef_dict = {
    "cx_net_coef": 1e-2,
    "cy_net_coef": 1e-3,
    "cz_net_coef": 1e-1,
    "trust_net_coef": 1e3,
}

net_params = {
    "cx_param_dim": len(cx_input_vars),
    "cy_param_dim": len(cy_input_vars),
    "cz_param_dim": len(cz_input_vars),
    "thrust_param_dim": len(thrust_input_vars),
    "regressor_layers": 3,
    "regressor_layer_dim": 64,
    "lr": 1e-5,
    "net_coef_dict": net_coef_dict,
    "equation_params": {
        "air_molar_mass": 29e-3,
        "gas_constant": 8.314,
        "wing_surface": 21.5,  # surface ailaire, en m^2
    },
}

saving_dir = Path.cwd() / "checkpoints" / training_name
log_dir = Path.cwd() / "logs"

callbacks = [
    ModelCheckpoint(
        saving_dir,
        "checkpoint.ckpt",
        monitor="val loss",
    )
]

trainer_params = {
    "max_epochs": 200,
    "callbacks": callbacks,
    "logger": TensorBoardLogger(save_dir=log_dir),
}

model = PhysicalModel(
    log_dir=log_dir,
    saving_dir=saving_dir,
    saving_name="checkpoint.ckpt",
    NetClass=FlightMecaNet3DEq,
    net_params=net_params,
    DatasetClass=GenericFlightMecaDatasetFast,
    dataset_params=dataset_params,
    trainer_params=trainer_params,
    batch_size=1024,
    num_loader_workers=8,  # à ajuster selon le nombre de coeurs dispos
)

In [ ]:
if not model._is_fitted:
    raise Exception("Le modèle doit d'abord être entraîné")

## Réalisation des prédictions sur les vols de test

In [ ]:
res = model.predict(data.test, concat_predict_and_data=True)

# Ajout du score d'anomalie
res = add_anomaly_score(res)

In [ ]:
print("Noms des vols et instants disponibles associés :")
for flight_name in res.index.get_level_values(0).unique():
    t_min = res.loc[flight_name].index.min()
    t_max = res.loc[flight_name].index.max()
    print(f"\tVol {flight_name}, instants disponibles de {t_min} à {t_max}.")

## Visualisation des résultats du modèle

### Évolution du score d'anomalie lors des vols

Visualisation de l'évolution du score d'anomalies lors des différents vols. La variable `anomaly_detection_quantile` permet de définir quelle quantile des données mettre en avant sur la figure.

In [ ]:
anomaly_detection_quantile = 0.95
fig = show_all_anomaly_scores(res, anomaly_detection_quantile)

### Exploration du score d'anomalies vol par vol

In [ ]:
flight_name = "vol 10"
start_time = 1.25
end_time = 280

fig = show_anomaly_score_single_flight(
    res,
    flight_name,
    start_time=start_time,
    end_time=end_time,
    anomaly_detection_quantile=anomaly_detection_quantile,
)
fig.show()

### Visualisation des résidus lors d'un vol

In [ ]:
flight_name = "vol 10"
start_time = 0
end_time = 299.95

fig = show_residue_evolution(
    res,
    flight_name,
    start_time,
    end_time,
)
fig.show()